# Applied Data Science by IBM

## Capstone project

In [1]:
#Install beautifulsoup
#!pip install beautifulsoup4
!pip install pgeocode

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [2]:
# importing necessary libraries
import numpy as np
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pgeocode
print("Libraries imported.")

Libraries imported.


In [3]:
#get Toronto neighborhood data on Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

In [4]:
#get postal code table from html
table_content = []
table = soup.find('table')
for row in table.findAll('td'):
    #print(row)
    cell = {}
    #print(row.span.text)
    #Drop postal codes that have no assigned neighborhoods
    if row.span.text == 'Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        rawNeighborh = (row.span.text).split('(')[1]
        #print(rawNeighborh)
        cell['Neighborhood'] = rawNeighborh.strip(')').replace('/',',').replace(')',',').strip(' ')
        table_content.append(cell)
        
#print(table_content)

In [5]:
#Create dataframe
toronto_df = pd.DataFrame(table_content)
toronto_df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."


In [6]:
#Clean up Borough data
toronto_df['Borough'] = toronto_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
toronto_df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."


In [7]:
#Get data dimentions
toronto_df.shape

(103, 3)

In [8]:
#Get gecoordinates
geolocator = pgeocode.Nominatim('ca')

for cell in zip(toronto_df['PostalCode']):
    postal_code = cell[0]
    location = geolocator.query_postal_code(postal_code)
    #print(postal_code, location.latitude, location.longitude)
    toronto_df.loc[toronto_df['PostalCode'] == postal_code, "Latitude"] = location.latitude
    toronto_df.loc[toronto_df['PostalCode'] == postal_code, "Longitude"] = location.longitude

toronto_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.7223,-79.4504
4,M7A,Queen's Park,Ontario Provincial Government,43.6641,-79.3889
5,M9A,Etobicoke,Islington Avenue,43.6662,-79.5282
6,M1B,Scarborough,"Malvern , Rouge",43.8113,-79.1930
7,M3B,North York,"Don Mills,North",43.7450,-79.3590
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.7063,-79.3094
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783
